In [24]:
import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import load_images
import data_set
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
FLAGS = dict()
FLAGS['width'] = 32
FLAGS['height'] = 32
FLAGS['batch_size'] = 16
FLAGS['kernel_1_out'] = 32
FLAGS['kernel_2_out'] = 16
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 5
FLAGS['train_report_step'] = 20
FLAGS['val_report_step'] = 80
FLAGS['keep_prob'] = 0.75
FLAGS['reg'] = 0.01


In [56]:
X_train, y_train = load_images.load_train(width = FLAGS['width'], 
                                                      height=FLAGS['height'])

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder NoF (Index: 4)
Load folder OTHER (Index: 5)
Load folder SHARK (Index: 6)
Load folder YFT (Index: 7)
Read train data time: 351.41 seconds


In [58]:
data = data_set.DataSet(X_train, y_train)

In [6]:
del X_train
del y_train

In [88]:
tf.add?

In [91]:
with tf.Graph().as_default() as g:
    with tf.name_scope('data'):
        X = tf.placeholder(tf.float32, [None, None, None, 3], name="X_placeholder")
        Y = tf.placeholder(tf.int32, [None], name="Y_placeholder")
        dropout = tf.placeholder(tf.float32, name='dropout')
        
    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    with tf.variable_scope('conv1') as scope:
        scope.reuse_variables()
        kernel1 = tf.Variable(tf.truncated_normal(shape=[FLAGS['width'], 
                                                        FLAGS['height'], 3, FLAGS['kernel_1_out']], 
                                                        name='kernel'))
        biases = tf.Variable(tf.random_normal(shape=[FLAGS['kernel_1_out']]), name="bias")
        conv = tf.nn.conv2d(X, kernel1, strides=[1, 1, 1, 1], padding='SAME')
        conv1 = tf.nn.relu(conv + biases, name=scope.name)
        kernel1_summ = tf.summary.scalar('kernel1_mean', tf.reduce_mean(kernel1))
        conv1_summ = tf.summary.scalar('conv1_mean', tf.reduce_mean(conv1))
   
    with tf.variable_scope('pool1') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME')
    
    with tf.variable_scope('conv2') as scope:
        kernel2 = tf.Variable(
            tf.truncated_normal(shape=[FLAGS['conv2_input_width'], 
                                       FLAGS['conv2_input_height'], 
                                       FLAGS['kernel_1_out'], 
                                       FLAGS['kernel_2_out']], name='kernel'))

        biases = tf.Variable(tf.random_normal(shape=[FLAGS['kernel_2_out']]), name="bias")
        conv = tf.nn.conv2d(pool1, kernel2, strides=[1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.relu(conv + biases, name=scope.name)
        kernel2_summ = tf.summary.scalar('kernel2_mean', tf.reduce_mean(kernel2))
        conv2_summ = tf.summary.scalar('conv2_mean', tf.reduce_mean(conv2))

    with tf.variable_scope('pool2') as scope:
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME') 
    
    with tf.variable_scope('fc') as scope:

        # reshape pool2 to 2 dimensional
        pool2 = tf.reshape(pool2, [FLAGS['batch_size'], -1])
        

        fcw = tf.Variable(tf.truncated_normal(shape=[1024, 128], name='kernel'))
        b = tf.Variable(tf.random_normal(shape=[128]), name="bias")
        
        fc = tf.nn.relu(tf.matmul(pool2, fcw) + b, name='relu')
        fcw_summ = tf.summary.scalar('fcw_mean', tf.reduce_mean(fcw))
        fc_summ = tf.summary.scalar('fc', tf.reduce_mean(fc))
        fc = tf.nn.dropout(fc, dropout, name='relu_dropout')
    
    with tf.variable_scope('softmax_linear') as scope:
        sfw = tf.Variable(tf.truncated_normal(shape=[128, FLAGS['n_classes']],name='weights'))
        softw_summ = tf.summary.scalar('softw_mean', tf.reduce_mean(sfw))
        b = tf.Variable(tf.random_normal(shape=[FLAGS['n_classes']], name='biases'))
        logits = tf.matmul(fc, sfw) + b

    with tf.name_scope('loss'):
        loss = tf.losses.sparse_softmax_cross_entropy(logits=logits, labels = Y)
        reg_los = 0
        reg_loss = 0.5 * FLAGS['reg'] * tf.reduce_sum(tf.square(kernel1))
        reg_loss += 0.5 * FLAGS['reg'] * tf.reduce_sum(tf.square(kernel2))
        reg_loss += 0.5 * FLAGS['reg'] * tf.reduce_sum(tf.square(fcw))
        reg_loss += 0.5 * FLAGS['reg'] * tf.reduce_sum(tf.square(sfw))
        loss += reg_loss
        loss_summ = tf.summary.scalar('loss', loss)
             
    optimizer = tf.train.AdamOptimizer(FLAGS['learning_rate']).minimize(loss, 
                                            global_step=global_step)

    merged = tf.summary.merge_all()
    print('hello')

    with tf.Session(graph=g) as sess:
        init = tf.global_variables_initializer()

        sess.run(init)

        saver = tf.train.Saver()
        #to visualize using TensorBoard
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('val_saver/'))
        # if that checkpoint exists, restore from checkpoint
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)

        initial_step = global_step.eval()

        start_time = time.time()

        n_batches = data.num_examples / FLAGS['batch_size']

        total_loss = 0.0
        last_val_loss = 0
        X_val, y_val = data.get_validation_set()
        val_patience = 1
        num_val_increases = 0
        train_writer = tf.summary.FileWriter('early_stopping_summary',
                                          sess.graph)
        train_writer.flush()
        val_writer.flush()
        for index in np.arange(initial_step, int(n_batches * FLAGS['n_epochs'])): # train the model n_epochs times
            X_batch, y_batch = data.next_batch(FLAGS['batch_size'])
            
            if (index + 1) % FLAGS['train_report_step'] == 0:
                _, loss_batch, summary, gs = sess.run([optimizer, loss, merged, global_step], 
                                    feed_dict={X: X_batch, Y:y_batch, dropout:FLAGS['keep_prob']}) 
                total_loss += loss_batch
                average_loss = total_loss / gs
                
                
                print('Average loss at step {}: {:5.1f}'.format(index + 1, average_loss))
                train_writer.add_summary(summary, index)
            else:
                _, loss_batch, gs = sess.run([optimizer, loss, global_step], 
                                    feed_dict={X: X_batch, Y:y_batch, dropout:FLAGS['keep_prob']}) 
                total_loss += loss_batch
                average_loss = total_loss / gs
                if average_loss < best_loss:
                    best_loss = average_loss
            
                
            if (index + 1) % FLAGS['val_report_step'] == 0:
                perm0 = np.arange(X_val.shape[0])
                np.random.shuffle(perm0)
                X_val= X_val[perm0]
                y_val = y_val[perm0]
                total_val_loss = 0
                print('checking validation loss...')
                for num in np.arange((X_val.shape[0] / FLAGS['batch_size']) - 1):
                    start_num = num * 16
                    end_num = int(start_num + 16)
                    l_val_batch = sess.run([loss], 
                                           feed_dict={X: X_val[int(start_num):end_num, :,:,:], Y: y_val[int(start_num):end_num], dropout:1.0}) 
                    total_val_loss += l_val_batch[0]    
                    
                avg_val_loss = total_val_loss / (X_val.shape[0] - FLAGS['batch_size'])
                print('validation loss: {}'.format(avg_val_loss))
                if avg_val_loss > last_val_loss:
                    print('validation loss increased')
                    num_val_increases += 1 
                    last_val_loss = avg_val_loss
                    if num_val_increases > val_patience:
                        print('Stopping training because validation loss has not decreased')
                        print('trained for {} batches'.format(index))
                        break
                else:
                    print('validation loss decreased...saving best params')
                    last_val_loss = avg_val_loss
                    num_val_increases = 0
                    saver.save(sess, 'early_stopping_saver/', gs)
                

hello
Average loss at step 20: 6854.5
Average loss at step 40: 5054.6
Average loss at step 60: 4223.8
Average loss at step 80: 3658.1
checking validation loss...
validation loss: 98.42107411977408
validation loss increased
Average loss at step 100: 3274.5
Average loss at step 120: 2998.0
Average loss at step 140: 2785.5
Average loss at step 160: 2618.2
checking validation loss...
validation loss: 83.79303258947424
validation loss decreased...saving best params
Average loss at step 180: 2477.4
Average loss at step 200: 2359.8
Average loss at step 220: 2257.9
Average loss at step 240: 2169.4
checking validation loss...
validation loss: 74.48021520666174
validation loss decreased...saving best params
Average loss at step 260: 2091.9
Average loss at step 280: 2023.7
Average loss at step 300: 1963.4
Average loss at step 320: 1909.6
checking validation loss...
validation loss: 69.58787990260768
validation loss decreased...saving best params
Average loss at step 340: 1861.3
Average loss at st